In [ ]:
%matplotlib inline
import os
import pandas as pd
import geopandas as gpd
import xarray as xr
import numpy as np
from scipy.stats import spearmanr
from itertools import combinations
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import seaborn as sns
from rasterio import features
from affine import Affine
import warnings
warnings.filterwarnings('ignore')


In [ ]:
def transform_from_latlon(lat, lon):
    lat = np.asarray(lat)
    lon = np.asarray(lon)
    trans = Affine.translation(lon[0], lat[0])
    scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
    return trans * scale

def rasterize(shapes, coords, fill=np.nan, **kwargs):
    """Rasterize a list of (geometry, fill_value) tuples onto the given
    xarray coordinates. This only works for 1d latitude and longitude
    arrays.
    """
    transform = transform_from_latlon(coords['lat'], coords['lon'])
    out_shape = (len(coords['lat']), len(coords['lon']))
    raster = features.rasterize(shapes, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float, **kwargs)
    return xr.DataArray(raster, coords=coords, dims=('lat', 'lon'))
    


In [ ]:
root_dir = "/dartfs-hpc/rc/lab/C/CMIG"
data_dir = os.path.join(root_dir, "agottlieb", "git_repos", "snow_drought", "data")
gridded_dir = os.path.join(data_dir,'gridded')
basin_dir = os.path.join(data_dir,'basin')
fig_dir = os.path.join(root_dir, "agottlieb", "git_repos","snow_drought", "figures")

In [ ]:
# get peak SWE from each gridded data product
if not os.path.exists(os.path.join(data_dir,'gridded_peak_ensemble.nc')):
    gridded_peak_ensemble = []
    for prod in os.listdir(gridded_dir):
        prod_dir = os.path.join(gridded_dir,prod)
        prod_files = [os.path.join(prod_dir,f) for f in os.listdir(prod_dir)]
        prod_files.sort()
        prod_peak = []
        for f in prod_files:
            with xr.open_dataset(f) as ds:
                prod_peak.append(ds['SWE'].resample(time='AS-OCT').max())
        prod_peak = xr.concat(prod_peak,dim='time')
        prod_peak = prod_peak.assign_coords(product=prod)
        gridded_peak_ensemble.append(prod_peak)
        print(prod)
    gridded_peak_ensemble = xr.concat(gridded_peak_ensemble,dim='product')
    gridded_peak_ensemble.name = 'peak_swe'
    gridded_peak_ensemble = gridded_peak_ensemble.to_dataset()
    gridded_peak_ensemble.to_netcdf(os.path.join(data_dir,'gridded_peak_ensemble.nc'))

In [ ]:
if not os.path.exists(os.path.join(data_dir,'gridded_corr.nc')):
    def pairwise_corr(X1,X2):
        try:
            r, p = spearmanr(X1,X2,nan_policy="omit")
            return r,p
        except:
            return np.nan, np.nan
        
    gridded_peak_ensemble = xr.open_dataset(os.path.join(data_dir,'gridded_peak_ensemble.nc'))
    product_combos = list(combinations(gridded_peak_ensemble["product"].values,2)) # generate all combinations of 2 products
    rs = []
    ps = []
    for combo in product_combos:
        X1 = gridded_peak_ensemble.sel(product=combo[0])['peak_swe']
        X2 = gridded_peak_ensemble.sel(product=combo[1])['peak_swe']
        r, p = xr.apply_ufunc(pairwise_corr,X1,X2,input_core_dims=[['time'],['time']],output_core_dims=[[],[]],vectorize=True)
        r = r.assign_coords(combo="+".join(combo))
        p = p.assign_coords(combo="+".join(combo))
        rs.append(r)
        ps.append(p)
        print(combo, "complete")
    rs = xr.concat(rs,dim='combo')
    ps = xr.concat(ps,dim='combo')
    rs.name = 'r'
    ps.name = 'p'
    corr_ds = xr.merge([rs,ps])
    corr_ds.to_netcdf(os.path.join(data_dir,'gridded_corr.nc'))

In [ ]:
# get peak SWE from each basin data product
if not os.path.exists(os.path.join(data_dir,'basin_peak_ensemble.nc')):
    basin_peak_ensemble = []
    for prod in os.listdir(basin_dir):
        prod_dir = os.path.join(basin_dir,prod)
        prod_files = [os.path.join(prod_dir,f) for f in os.listdir(prod_dir)]
        prod_files.sort()
        prod_peak = []
        for f in prod_files:
            with xr.open_dataset(f) as ds:
                prod_peak.append(ds['SNOMAS'].resample(time='AS-OCT').max())
        prod_peak = xr.concat(prod_peak,dim='time')
        prod_peak = prod_peak.assign_coords(product=prod)
        basin_peak_ensemble.append(prod_peak)
        print(prod)
    basin_peak_ensemble = xr.concat(basin_peak_ensemble,dim='product')
    basin_peak_ensemble.name = 'peak_swe'
    basin_peak_enesemble = basin_peak_ensemble.to_dataset()
    basin_peak_ensemble.to_netcdf(os.path.join(data_dir,'basin_peak_ensemble.nc'))

In [ ]:
if not os.path.exists(os.path.join(data_dir,'basin_corr.nc')):
    def pairwise_corr(X1,X2):
        try:
            r, p = spearmanr(X1,X2,nan_policy="omit")
            return r,p
        except:
            return np.nan, np.nan
        
    basin_peak_ensemble = xr.open_dataset(os.path.join(data_dir,'basin_peak_ensemble.nc'))
    product_combos = list(combinations(basin_peak_ensemble["product"].values,2)) # generate all combinations of 2 products
    rs = []
    ps = []
    for combo in product_combos:
        X1 = basin_peak_ensemble.sel(product=combo[0])['peak_swe']
        X2 = basin_peak_ensemble.sel(product=combo[1])['peak_swe']
        r, p = xr.apply_ufunc(pairwise_corr,X1,X2,input_core_dims=[['time'],['time']],output_core_dims=[[],[]],vectorize=True)
        r = r.assign_coords(combo="+".join(combo))
        p = p.assign_coords(combo="+".join(combo))
        rs.append(r)
        ps.append(p)
        print(combo, "complete")
    rs = xr.concat(rs,dim='combo')
    ps = xr.concat(ps,dim='combo')
    rs.name = 'r'
    ps.name = 'p'
    corr_ds = xr.merge([rs,ps])
    corr_ds.to_netcdf(os.path.join(data_dir,'basin_corr.nc'))

In [ ]:
gridded_peak_ensemble = xr.open_dataset(os.path.join(data_dir,'gridded_peak_ensemble.nc'))
gridded_peak_ensemble = gridded_peak_ensemble.where(gridded_peak_ensemble['peak_swe']>0)
gridded_peak_ensemble = gridded_peak_ensemble.where(gridded_peak_ensemble["peak_swe"].count(("product","time"))>72)

# calculate ensemble spread-to-mean ratio at each timestep
gridded_peak_max = gridded_peak_ensemble["peak_swe"].max("product")
gridded_peak_min = gridded_peak_ensemble["peak_swe"].min("product")
gridded_peak_mean = gridded_peak_ensemble["peak_swe"].mean("product")
gridded_ens_uncert = (gridded_peak_max-gridded_peak_min) / gridded_peak_mean
gridded_ens_uncert = gridded_ens_uncert.where(gridded_ens_uncert.max("time")!=0)

In [ ]:
basin_peak_ensemble = xr.open_dataset(os.path.join(data_dir,'basin_peak_ensemble.nc'))
basin_peak_ensemble = basin_peak_ensemble.where(basin_peak_ensemble['peak_swe']>0)
basin_peak_ensemble = basin_peak_ensemble.where(basin_peak_ensemble["peak_swe"].count(("product","time"))>72)

# calculate ensemble spread-to-mean ratio at each timestep
basin_peak_max = basin_peak_ensemble["peak_swe"].max("product")
basin_peak_min = basin_peak_ensemble["peak_swe"].min("product")
basin_peak_mean = basin_peak_ensemble["peak_swe"].mean("product")
basin_ens_uncert = (basin_peak_max-basin_peak_min) / basin_peak_mean
basin_ens_uncert = basin_ens_uncert.where(basin_ens_uncert.max("time")!=0)

# rasterize basin values for plotting
basin_ens_uncert = basin_ens_uncert.to_dataframe().reset_index()
grdc_basins = gpd.read_file(os.path.join(data_dir, "boundaries", "grdc_basins"), layer="mrb_basins")
basin_ens_uncert = grdc_basins[['geometry']].merge(basin_ens_uncert,left_index=True,right_on='basin')
basin_ens_uncert = rasterize(zip(basin_ens_uncert.geometry,basin_ens_uncert.peak_swe),gridded_peak_ensemble.drop(["time","product"]).coords)


In [ ]:
gridded_corr = xr.open_dataset(os.path.join(data_dir,'gridded_corr.nc'))
gridded_corr_mean = gridded_corr['r'].mean("combo").where(~xr.ufuncs.isnan(gridded_ens_uncert.mean("time")))
gridded_frac_sig = (gridded_corr['p'] < 0.05).astype(int).sum("combo") / gridded_corr['p'].count("combo")
gridded_frac_sig = gridded_frac_sig.where(~xr.ufuncs.isnan(gridded_ens_uncert.mean("time")))


In [ ]:
basin_corr = xr.open_dataset(os.path.join(data_dir,'basin_corr.nc'))
basin_corr_mean = basin_corr['r'].mean("combo")
basin_frac_sig = (basin_corr['p'] < 0.05).astype(int).sum("combo") / basin_corr['p'].count("combo")


In [ ]:
basin_corr_mean = basin_corr_mean.to_dataframe().reset_index()
basin_corr_mean = grdc_basins[['geometry']].merge(basin_corr_mean,left_index=True,right_on='basin')
basin_corr_mean = rasterize(zip(basin_corr_mean.geometry,basin_corr_mean.r),gridded_peak_ensemble.drop(["time","product"]).coords)
basin_corr_mean = basin_corr_mean.where(~xr.ufuncs.isnan(basin_ens_uncert))

In [ ]:
basin_frac_sig = basin_frac_sig.to_dataframe().reset_index()
basin_frac_sig = grdc_basins[['geometry']].merge(basin_frac_sig,left_index=True,right_on='basin')
basin_frac_sig = rasterize(zip(basin_frac_sig.geometry,basin_frac_sig.p),gridded_peak_ensemble.drop(["time","product"]).coords)
basin_frac_sig = basin_frac_sig.where(~xr.ufuncs.isnan(basin_ens_uncert))

In [ ]:
basin_corr = xr.open_dataset(os.path.join(data_dir,'basin_corr.nc'))
basin_corr_mean = basin_corr['r'].mean("combo").where(~xr.ufuncs.isnan(basin_ens_uncert))
basin_frac_sig = (basin_corr['p'] < 0.05).astype(int).sum("combo") / basin_corr['p'].count("combo")
basin_frac_sig = basin_frac_sig.where(~xr.ufuncs.isnan(basin_ens_uncert))

basin_corr_mean = basin_corr_mean.to_dataframe().reset_index()
basin_corr_mean = grdc_basins[['geometry']].merge(basin_corr_mean,left_index=True,right_on='basin')
basin_corr_mean = rasterize(zip(basin_corr_mean.geometry,basin_corr_mean.r),gridded_peak_ensemble.drop(["time","product"]).coords)

basin_frac_sig = basin_frac_sig.to_dataframe().reset_index()
basin_frac_sig = grdc_basins[['geometry']].merge(basin_frac_sig,left_index=True,right_on='basin')
basin_frac_sig = rasterize(zip(basin_frac_sig.geometry,basin_frac_sig.p),gridded_peak_ensemble.drop(["time","product"]).coords)


In [ ]:
basin_ids = {k: i for i, k in enumerate(grdc_basins.RIVER_BASI)}
basin_shapes = [(shape, n) for n, shape in enumerate(grdc_basins.geometry)]

In [ ]:
# load basin ensemble
basin_peak_ensemble = basin_peak_ensemble.to_dataframe().reset_index()

# subset out same years of common overlap
basin_peak_ensemble["time"] = pd.to_datetime(basin_peak_ensemble["time"])
basin_peak_ensemble = basin_peak_ensemble[(basin_peak_ensemble.time.dt.date>=pd.to_datetime("2002-10-01")) & (basin_peak_ensemble.time.dt.date<=pd.to_datetime("2017-10-01"))]

# calculate ensemble spread-to-mean ratio
basin_peak_ensemble["peak_swe"] = basin_peak_ensemble["peak_swe"].replace(0,np.nan)
basin_peak_stats = basin_peak_ensemble.groupby(["basin","time"]).agg({"peak_swe": ["max","min", "mean"]})
basin_peak_spread = (basin_peak_stats["peak_swe","max"]-basin_peak_stats["peak_swe","min"]) / basin_peak_stats["peak_swe","mean"]
basin_peak_spread.name="ensemble_uncertainty"

# merge with basins shapefile
basin_ens_uncert = basin_peak_spread.reset_index().groupby("basin").agg({"ensemble_uncertainty": "mean"})
basin_ens_uncert = gpd.GeoDataFrame(basin_ens_uncert.merge(grdc_basins[["geometry"]],left_index=True,right_index=True))


In [ ]:
# load stats of pairwise correlation distributions
gridded_r_ds = xr.open_dataset(os.path.join(data_dir,"ensembles","gridded_peak_spearmanr.nc"))
gridded_p_ds = xr.open_dataset(os.path.join(data_dir,"ensembles","gridded_peak_spearmanr_p.nc"))

# gridded_r_ds = xr.open_dataset(os.path.join(data_dir,"ensembles","peak_spearmanr_all_prods.nc"))
r_mean = gridded_r_ds['peak_swe'].mean("combo").where(~xr.ufuncs.isnan(gridded_ens_uncert.mean("time")))
frac_sig = (gridded_p_ds['peak_swe'] < 0.05).astype(int).sum("combo") / gridded_p_ds['peak_swe'].count("combo")
frac_sig = frac_sig.where(~xr.ufuncs.isnan(gridded_ens_uncert.mean("time")))

basin_r = gpd.read_file(os.path.join(data_dir,"ensembles","basin_peak_spearmanr_all_products"))
basin_r.set_index("index",inplace=True)

# rasterize basin stats
basin_r_ds = rasterize(zip(basin_r.geometry,basin_r.r_mean),gridded_peak_ensemble.drop(["time","product"]).coords)
basin_r_sig = rasterize(zip(basin_r.geometry,basin_r.frac_sig),gridded_peak_ensemble.drop(["time","product"]).coords)

In [ ]:
# get boundaries to indicate different numbers of products
world_borders = gpd.read_file(os.path.join(data_dir,'boundaries',"world_borders"))
na_borders = world_borders[world_borders.ISO3.isin(["USA","MEX","CAN"])].dissolve(by="REGION") # North America (10)

us_borders = gpd.read_file(os.path.join(data_dir,"boundaries","us_states"))
conus_borders = us_borders[~us_borders.STUSPS.isin(["AK","HI","MP","AS","PR","VI","GU"])].dissolve(by="FUNCSTAT") # CONUS (15)

In [ ]:
keep_basins = basin_corr['r'].where(~xr.ufuncs.isnan(basin_corr['r'].mean("combo")),drop='True')['basin']

In [ ]:
sns.set(font_scale=2, style="white")
mpl.rcParams['hatch.linewidth'] = 2
fig = plt.figure(figsize=(34,13.6))
gs = gridspec.GridSpec(ncols=2, nrows=2, figure=fig)

# binned colormap for uncertainty in magnitude
mag_cmap = plt.cm.get_cmap("RdBu_r")
mag_bounds = [1/5,1/4,1/3,1/2,1,2,3,4,5]
mag_colors = [mag_cmap(x) for x in [0.1,0.175,0.25,0.325,0.675,0.75,0.825,0.9]]
under = mag_cmap(0)
over = mag_cmap(0.99)
mag_cmap = mpl.colors.ListedColormap(mag_colors,name="mag_cmap")
mag_cmap.set_under(under)
mag_cmap.set_over(over)
mag_norm = mpl.colors.BoundaryNorm(boundaries=mag_bounds,ncolors=8)

# binned colormap for correlations
corr_cmap = plt.cm.get_cmap("Reds_r", 10)
corr_norm = mpl.colors.BoundaryNorm(np.arange(0,1.01,0.1),11)

# plot grid-cell uncertainty
ax1 = plt.subplot(gs[0,0],projection=ccrs.Robinson())
ax1.set_aspect("equal","box")
ax1.set_extent([-180,180,0,90], ccrs.PlateCarree())
gridded_ens_uncert.where(~xr.ufuncs.isnan(gridded_corr_mean)).mean("time").plot(ax=ax1, transform=ccrs.PlateCarree(), cmap=mag_cmap,norm=mag_norm, add_colorbar=False)
asp = ax1.get_aspect()
na_borders.geometry.boundary.plot(ax=ax1,transform=ccrs.PlateCarree(),color="indigo") # outline regions with different numbers of products
conus_borders.geometry.boundary.plot(ax=ax1,transform=ccrs.PlateCarree(),color="green",linewidth=2)
us_borders[us_borders.STUSPS=="HI"].geometry.boundary.plot(ax=ax1,transform=ccrs.PlateCarree(),color="black")
ax1.set_aspect(asp)

# plot basin-scale uncertainty
ax3 = plt.subplot(gs[1,0],projection=ccrs.Robinson())
basin_ens_uncert.where(~xr.ufuncs.isnan(basin_corr_mean)).plot(ax=ax3, transform=ccrs.PlateCarree(), cmap=mag_cmap,norm=mag_norm, add_colorbar=False)
grdc_basins.iloc[keep_basins].geometry.boundary.plot(ax=ax3, transform=ccrs.PlateCarree(), color="black", lw=0.5) # add basin_outlines


# plot grid-cell mean of pairwise correlation distribution
ax2 = plt.subplot(gs[0,1],projection=ccrs.Robinson())
gridded_corr_mean.plot(ax=ax2, transform=ccrs.PlateCarree(), cmap=corr_cmap,norm=corr_norm, add_colorbar=False)
gridded_frac_sig.plot.contourf(ax=ax2,transform=ccrs.PlateCarree(), colors="none",levels=[0,1/2,2],hatches=["\\\\",None],add_colorbar=False) # add hatching where fewer than half of correlations significant

# plot basin-scale uncertainty
ax4 = plt.subplot(gs[1,1],projection=ccrs.Robinson())
basin_corr_mean.plot(ax=ax4, transform=ccrs.PlateCarree(), cmap=corr_cmap,norm=corr_norm, add_colorbar=False)
basin_frac_sig.plot.contourf(ax=ax4, transform=ccrs.PlateCarree(), colors="none",levels=[0,1/2,2],hatches=["\\\\",None],add_colorbar=False)  # add hatching where fewer than half of correlations significant
grdc_basins.iloc[keep_basins].geometry.boundary.plot(ax=ax4, transform=ccrs.PlateCarree(), color="black", lw=0.5) # add basin_outlines


# add colorbar for uncertainty in magnitude
mag_sm = plt.cm.ScalarMappable(cmap=mag_cmap, norm=mag_norm)
mag_sm.set_array([])
mag_cbar = fig.colorbar(mag_sm, ax=[ax1, ax3], orientation="horizontal", ticks=mag_bounds, pad=0.05,use_gridspec=True, extend="both")
mag_cbar.ax.set_xlabel(r"Ensemble spread-to-mean ratio", labelpad=10)
mag_cbar.ax.set_xticklabels(["1/5","1/4","1/3","1/2","1/1","2/1","3/1","4/1","5/1"])
fig.text(0.13,0.79, "a",fontsize=30,fontweight="bold")
fig.text(0.13,0.46, "c",fontsize=30,fontweight="bold")

# add colorbar for average correlation
corr_sm = plt.cm.ScalarMappable(cmap=corr_cmap, norm=corr_norm)
corr_sm.set_array([])
corr_cbar = fig.colorbar(corr_sm, ax=[ax2, ax4], orientation="horizontal", ticks = np.arange(0,1.01,0.1), pad=0.05,use_gridspec=True)
corr_cbar.ax.set_xlabel(r"Mean pairwise Spearman's $\rho$", labelpad=10)
corr_cbar.ax.set_xticklabels([np.round(0.1*i,1) for i in range(11)])
fig.text(0.552,0.79, "b",fontsize=30,fontweight="bold")
fig.text(0.552,0.46, "d",fontsize=30,fontweight="bold")

fig.text(0.1, 0.73, "Grid cell", fontsize=28, rotation=90, va="center")
fig.text(0.1, 0.4, "Basin", fontsize=28, rotation=90, va="center")
fig.text(0.3, 0.84, "Average Ensemble Uncertainty in Peak SWE", fontsize=28, ha="center")
fig.text(0.73, 0.84, "Ensemble Agreement on Temporal Ranks of Peak SWE", fontsize=28, ha="center")

for ax in [ax1, ax2, ax3, ax4]:
    ax.set_extent([-180,180,10,85], ccrs.PlateCarree())
    ax.coastlines('110m')
    ax.add_feature(cartopy.feature.OCEAN, facecolor="white")
    ax.add_feature(cartopy.feature.LAKES, facecolor="grey",alpha=0.1)
    ax.title.set_text("")
    ax.set_aspect("equal","box")
    ax.add_feature(cartopy.feature.LAND, facecolor="grey",alpha=0.3)

plt.savefig(os.path.join(fig_dir, "fig2.pdf"),bbox_inches='tight')